In [1]:
# Load IPFX libraries
from ipfx.dataset.create import create_ephys_data_set
from ipfx.data_set_features import extract_data_set_features
from ipfx.utilities import drop_failed_sweeps

# Load pandas library too
import pandas as pd
mouse_data_dict = {}

In [11]:
from ipfx.data_access import get_archive_info
archive_url, file_manifest, experiment_metadata = get_archive_info(dataset="mouse")
mouse_specimen_id = list(experiment_metadata['cell_specimen_id'])
file_manifest
file_name_dict = {}
for index, row in file_manifest.iterrows():
    if row["file_name"][-4:] == ".nwb":
        file_name_dict[row["cell_specimen_id"]] = row["file_name"]
        
new_file_name = {}
index = 0
for item in file_name_dict:
    if 345 < index < 408:
        new_file_name[item] = file_name_dict[item]
    index += 1

file_name_dict = new_file_name

In [12]:
file_name_dict

{653191401.0: 'sub-650125704_ses-653191386_icephys.nwb',
 653195941.0: 'sub-652361194_ses-653195922_icephys.nwb',
 653196359.0: 'sub-652361194_ses-653196344_icephys.nwb',
 653199987.0: 'sub-652361164_ses-653199958_icephys.nwb',
 653203361.0: 'sub-652361164_ses-653203346_icephys.nwb',
 653232374.0: 'sub-652361164_ses-653232180_icephys.nwb',
 653289801.0: 'sub-652361331_ses-653289786_icephys.nwb',
 653326186.0: 'sub-652361331_ses-653326170_icephys.nwb',
 653326966.0: 'sub-652361331_ses-653326944_icephys.nwb',
 653327067.0: 'sub-652361331_ses-653327052_icephys.nwb',
 653327513.0: 'sub-652361331_ses-653327497_icephys.nwb',
 653327546.0: 'sub-652361331_ses-653327528_icephys.nwb',
 653677436.0: 'sub-652361174_ses-653677241_icephys.nwb',
 653699672.0: 'sub-652361174_ses-653699552_icephys.nwb',
 653744562.0: 'sub-652361174_ses-653744209_icephys.nwb',
 653795917.0: 'sub-652361291_ses-653795878_icephys.nwb',
 653832102.0: 'sub-652361291_ses-653832048_icephys.nwb',
 654294107.0: 'sub-653545528_se

In [3]:
def summarize_cell_ephys_features(lsa_results):
    hero_keep_features = ['adapt', 'avg_rate', 'first_isi', 'isi_cv', 'latency', 'mean_isi', 'median_isi', 
                          'stim_amp']
    rheo_keep_features = ['threshold_v', 'peak_v', 'trough_v', 
     'fast_trough_v', 'adp_v', 'width', 'upstroke_downstroke_ratio', 'peak_t', 'fast_trough_t', 'trough_t']
    
    overall_cell_keep_features = ['v_baseline', 'rheobase_i', 'fi_fit_slope', 
                                  'sag', 'vm_for_sag', 'input_resistance', 'tau']
    
    hero_small_dict = lsa_results['hero_sweep'][hero_keep_features]
    rheobase_sweep_index = lsa_results['rheobase_sweep'].name
    rheobase_sweep = lsa_results['spikes_set'][rheobase_sweep_index].iloc[0]
    
    rheo_spike_small_dict = rheobase_sweep[rheo_keep_features]
    rheo_first_isi = lsa_results['rheobase_sweep']['first_isi']
    rheo_spike_small_dict['rheo_first_isi'] = rheo_first_isi

    spike_comb_dict = {**hero_small_dict, **rheo_spike_small_dict}

    overall_cell_features = {x: lsa_results[x] for x in overall_cell_keep_features if x in lsa_results}
    final_cell_feature_dict = {**spike_comb_dict, **overall_cell_features}
    return(final_cell_feature_dict)

In [ ]:
import urllib.request
from ipfx.dataset.create import create_ephys_data_set
from ipfx.utilities import drop_failed_sweeps

for item in file_name_dict:
    specimen_id = item
    file_name_spec = file_name_dict[item]
#     row = file_manifest.query('cell_specimen_id == {} & technique == "intracellular_electrophysiology"'.format(specimen_id))
#     archive_uri = row['archive_uri'].values[0]
#     file_name = row['file_name'].values[0]
#     print('Intrinsic ephys file url: ' + archive_uri)
#     print('Intrinsic ephys file name: ' + file_name)

#     print('downloading data for cell from DANDI...')
    try:
#         urllib.request.urlretrieve(archive_uri, file_name)
#         print('data downloaded!')

        # Create Ephys Data Set

        print('loading dataset into data structure...')
        nwb_file_name = '/external/rprshnas01/netdata_kcni/stlab/AIBS_patchseq_2020/mouse/ephys/000020/{}'.format(file_name_spec)
        data_set = create_ephys_data_set(nwb_file=nwb_file_name) ##loads nwb file into ipfx data structure

        # Drop failed sweeps: sweeps with incomplete recording or failing QC criteria
        drop_failed_sweeps(data_set)

        from ipfx.feature_extractor import SpikeFeatureExtractor, SpikeTrainFeatureExtractor

        import ipfx.stimulus_protocol_analysis as spa
        from ipfx.epochs import get_stim_epoch
        import matplotlib.pyplot as plt


        # get sweep table of Long Square sweeps
        long_square_table = data_set.filtered_sweep_table(
            stimuli=data_set.ontology.long_square_names
        )
        long_square_sweeps = data_set.sweep_set(long_square_table.sweep_number)

        # Select epoch corresponding to the actual recording from the sweeps
        # and align sweeps so that the experiment would start at the same time
        long_square_sweeps.select_epoch("recording")
        long_square_sweeps.align_to_start_of_epoch("experiment")

        # find the start and end time of the stimulus
        # (treating the first sweep as representative)
        stim_start_index, stim_end_index = get_stim_epoch(long_square_sweeps.i[0])
        stim_start_time = long_square_sweeps.t[0][stim_start_index]
        stim_end_time = long_square_sweeps.t[0][stim_end_index]

        # build the extractors
        spfx = SpikeFeatureExtractor(start=stim_start_time, end=stim_end_time)
        sptfx = SpikeTrainFeatureExtractor(start=stim_start_time, end=stim_end_time)

        # run the analysis and print out a few of the features
        long_square_analysis = spa.LongSquareAnalysis(spfx, sptfx, subthresh_min_amp=-100.0)
        data = long_square_analysis.analyze(long_square_sweeps)

        specimen_dict = summarize_cell_ephys_features(data)
        mouse_data_dict[specimen_id] = specimen_dict
        
        print("{} extracted".format(item))
        dfObj = pd.DataFrame(mouse_data_dict)
        dfObj.to_csv('mouse_features_from_nwb8.csv')
        
    except urllib.error.HTTPError as err:
        print(err.code)

